## 1. 导入库与路径配置

In [11]:
import os
import yaml
import numpy as np
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial.transform import Rotation as R

# ================= 路径配置 =================
# 请修改为你自己的路径
DATA_ROOT = "/home/xzy/datasets/Rellis-3D" 
SEQ = "00004"
FRAME_ID_STR = "000000"  # 字符串形式的帧ID

# 1. 图像路径
IMG_PATH = os.path.join(DATA_ROOT, SEQ, 'pylon_camera_node', f'frame{FRAME_ID_STR}-1581791678_408.jpg') # 注意：这里文件名可能需要根据实际情况修改通配符或具体名称

# 2. 点云路径
BIN_PATH = os.path.join(DATA_ROOT, SEQ, 'os1_cloud_node_kitti_bin', f'{FRAME_ID_STR}.bin')

# 3. 位姿路径 (LiDAR在世界系下的位姿)
POSES_PATH = os.path.join(DATA_ROOT, SEQ, 'poses.txt')

# 4. 标定文件路径 (Camera 到 LiDAR 的外参)
CALIB_PATH = os.path.join(DATA_ROOT, SEQ, 'transforms.yaml')

print("Checking paths:")
print(f"Img: {os.path.exists(IMG_PATH)}")
print(f"Bin: {os.path.exists(BIN_PATH)}")
print(f"Poses: {os.path.exists(POSES_PATH)}")
print(f"Calib: {os.path.exists(CALIB_PATH)}")

Checking paths:
Img: True
Bin: True
Poses: True
Calib: True


## 2. 数据读取与矩阵处理工具函数

In [16]:
def load_lidar_bin(bin_path):
    """读取二进制点云 (x, y, z, i)"""
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return points[:, :3] # 只取 XYZ

def read_poses(filename):
    """读取 poses.txt (T_world_lidar)"""
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

def get_transform_from_yaml(yaml_path, parent_frame, child_frame):
    """
    解析 ROS 风格的 transforms.yaml
    
    针对类似 Rellis-3D 的变换格式，其中包含多个 frame 变换定义
    格式示例：
    parent_frame-child_frame:
      q: {w: 0.5, x: 0.5, y: 0.5, z: 0.5}
      t: {x: 1.0, y: 2.0, z: 3.0}
    
    返回 4x4 矩阵 T_parent_child
    """
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    
    # 方法1: 尝试使用提供的 frame 名称组合作为键
    frame_key = f"{parent_frame}-{child_frame}"
    if frame_key in data:
        transform_data = data[frame_key]
    else:
        # 方法2: 查找包含指定 frame 名称的键
        matching_keys = [key for key in data.keys() 
                        if parent_frame in key and child_frame in key]
        if not matching_keys:
            print(f"Error: No transform found for {parent_frame}->{child_frame}")
            return np.eye(4)
        # 如果有多个匹配，使用第一个
        frame_key = matching_keys[0]
        transform_data = data[frame_key]
    
    # 解析平移向量
    t_data = transform_data.get('t', {})
    t = np.array([
        t_data.get('x', 0.0),
        t_data.get('y', 0.0), 
        t_data.get('z', 0.0)
    ])
    
    # 解析四元数旋转
    q_data = transform_data.get('q', {})
    # ROS 四元数顺序: (x, y, z, w)
    q = np.array([
        q_data.get('x', 0.0),
        q_data.get('y', 0.0),
        q_data.get('z', 0.0),
        q_data.get('w', 1.0)  # 默认 w=1.0
    ])
    
    # 构建变换矩阵
    T = np.eye(4)
    if np.linalg.norm(q) > 0:  # 检查四元数是否有效
        q = q / np.linalg.norm(q)  # 归一化
        R_mat = R.from_quat(q).as_matrix()
        T[:3, :3] = R_mat
    T[:3, 3] = t
    
    print(f"Loaded transform {frame_key}: t={t}, q={q}")
    return T

# ================= 相机内参 (Intrinsics) =================
# Rellis-3D 的相机内参通常不包含在 transforms.yaml 中，而是单独的 camera_info
# 如果你没有内参文件，这里提供一个 Rellis-3D 常用的近似值 (需根据实际情况微调)
# 假设图片分辨率是 1920 x 1200 或类似
def get_camera_intrinsics():
    # 示例内参，如果投影偏差大，需要替换为真实 K
    # fx, fy, cx, cy
    K = np.array([
        [1395.0,    0.0, 960.0],
        [   0.0, 1395.0, 600.0],
        [   0.0,    0.0,   1.0]
    ])
    return K

K = get_camera_intrinsics()
print("Using Camera Intrinsics:\n", K)

T_l_c = get_transform_from_yaml(CALIB_PATH, parent_frame='os1_cloud_node', child_frame='pylon_camera_node') 
print("Extrinsic T_lidar_camera:\n", T_l_c)

Using Camera Intrinsics:
 [[1.395e+03 0.000e+00 9.600e+02]
 [0.000e+00 1.395e+03 6.000e+02]
 [0.000e+00 0.000e+00 1.000e+00]]
Loaded transform os1_cloud_node-pylon_camera_node: t=[-0.09581321  0.04062246 -0.16507774], q=[ 0.5115674   0.49051429 -0.49270892 -0.50490835]
Extrinsic T_lidar_camera:
 [[ 0.0332673   0.00431654 -0.99943717 -0.09581321]
 [ 0.99940794 -0.00892657  0.03322777  0.04062246]
 [-0.00877812 -0.99995084 -0.00461095 -0.16507774]
 [ 0.          0.          0.          1.        ]]


## 3. 可视化雷达与相机的双轨迹

基本确定，雷达系前左上为xyz，相机系左下后为xyz，因为相机朝后，而实际的运动正好是往后运动。

In [ ]:
# 1. 读取雷达位姿 (T_w_l)
lidar_poses = read_poses(POSES_PATH)

# 2. 读取标定外参 (T_l_c: Camera 到 LiDAR)
# 注意：你的描述说 yaml 描述的是 "相机相对于雷达"，即 T_lidar_camera (点从 Cam 转到 Lidar)
# 通常 child: camera, parent: lidar


# 3. 计算相机在世界系下的位姿 (T_w_c)
# T_w_c = T_w_l * T_l_c
camera_poses = []
for T_w_l in lidar_poses:
    T_w_c = T_w_l @ T_l_c
    camera_poses.append(T_w_c)

# 4. 可视化函数
def visualize_dual_trajectory(poses_a, poses_b, step=20):
    geoms = []
    
    # 轨迹线 A (LiDAR) - 黄色
    pts_a = [p[:3, 3] for p in poses_a]
    lines_a = [[i, i+1] for i in range(len(pts_a)-1)]
    ls_a = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(pts_a),
        lines=o3d.utility.Vector2iVector(lines_a)
    )
    ls_a.paint_uniform_color([1, 0.8, 0]) # Yellow
    geoms.append(ls_a)
    
    # 轨迹线 B (Camera) - 青色
    pts_b = [p[:3, 3] for p in poses_b]
    lines_b = [[i, i+1] for i in range(len(pts_b)-1)]
    ls_b = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(pts_b),
        lines=o3d.utility.Vector2iVector(lines_b)
    )
    ls_b.paint_uniform_color([0, 0.8, 1]) # Cyan
    geoms.append(ls_b)
    
    # 坐标轴可视化
    # 对于 Camera：红色=右, 绿色=下, 蓝色=前 (光学坐标系) 
    # 或者 红色=前, 绿色=左, 蓝色=上 (如果是ROS坐标系)
    # 观察蓝色轴的指向非常重要！
    for i in range(0, len(poses_a), step):
        # LiDAR 坐标轴 (大)
        axis_l = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0)
        axis_l.transform(poses_a[i])
        geoms.append(axis_l)
        
        # Camera 坐标轴 (小)
        axis_c = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.5)
        axis_c.transform(poses_b[i])
        geoms.append(axis_c)
        
    o3d.visualization.draw_geometries(geoms, window_name="LiDAR (Yellow) vs Camera (Cyan) Trajectory")

print("Visualizing Trajectories... Close window to continue.")
# 仅显示前 300 帧以防卡顿
visualize_dual_trajectory(lidar_poses[:300], camera_poses[:300], step=50)

Visualizing Trajectories... Close window to continue.
